# Create and Train our model 

The following steps are:
    -Create csv file "data_CSVFile.csv"
    - Load Data and normalize images 
    - split data into 80% training and 20% testing 
    - Encode labels 
    - Build a CNN architecture with multi_output
    - Choose model compilation parameters(optimizer, lossFunction,metrics,..)
    - Fit the model with Nb_epochs and a defined batch size,..
    - save the model 
    - Display histograms
   

In [ ]:
#First of all we have to import necessary librairies
from config import *
#Config is our configuration file 
import resize_img_bb
from keras .layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Input 
from keras.layers import Convolution2D as conv
from keras.layers import MaxPooling2D as maxpool
from keras.layers import Flatten as flt
from keras.models import Model 
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical 
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt 
import numpy as np
import pickle 
import cv2
import os
import pandas as pd

In [ ]:
def prepare_data():
    
    print("[INFO] resizing images and extracting bounding boxes...")
    
    csvFile = resize_img_bb.annotations_to_csvFile(ANNOTS_PATH, IMAGES_PATH, WRITE_NEW_IMAGES_PATH,
                                                   CLASSES_DICT, IMAGE_SIZE)
    
    print("[INFO]loading dataset...")
    data =[]
    lables = []
    bboxs=[]
    img_paths=[]
    dataset = pd.read_csv(csvFile)
    
    for index,row in dataset.iterrows():
        image = cv2.imread(row["new_path"])
        coordinates= row['new_bb'][1:-1].split(',')
        Xmin = float(coordinates[0])
        Ymin = float(coordinates[1])
        Xmax = float(coordinates[2])
        Ymax = float(coordinates[3])
        image = img_to_array(image)
        data.append(image)
        labels.append(row["class"])
        bboxs.append((Xmin,Ymin,Xmax,Ymax))
        img_paths.append(row["new_path"])
    data= np.array(data,stype ="float32")/255.0
    labels = np.array(labels)
    bboxes = np.array(bboxs,dtype="float32")
    img_paths = np.array(img_paths)
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    
    split = train_test_split(data, labels,bboxes, filenames, test_size=0.20,random_state=42)
    (train_images,test_images) = split[:,:2]
    (train_labels,test_labels) = split[:,2:4]
    (train_bboxes,test_bboxes) =split[:,4:6]
    (train_filenames,test_filenames)=split[:,6:]
    if(lb.classes_ == 2):
        labels = to_categorical(labels)
    return (data,bboxes,img_paths,labels,len(lb.classes_))

In [ ]:
def create_MultiOutput_Model(nbClasses,img_size):
    print("[INFO] constructing model architecture...")
    print("[INFO] input shape : ("+str(img_size)+","+str(img_size)+",3)...")
    
    model = conv(64,(3,3),input_shape=(img_size,img_size,3),activation='relu',padding = 'same', strides=(1,1))
    model = conv(64,(3,3),activation='relu',padding = 'same', strides=(1,1)) (model)
    model = maxpool(pool_size=(2,2),strides=(2,2),padding='same')(model)
    print("2 x CONV  of 64 filters  and (3,3) kernel size...")
    print("Max pool of (2,2) kernel size...")
    model = conv(128,(3,3),activation='relu',padding = 'same', strides=(1,1)) (model)
    model = maxpool(pool_size=(2,2),strides=(2,2),padding='same')(model)
    print("CONV  of 128 filters  and (3,3) kernel size...")
    print("Max pool of (2,2) kernel size...")
    model = conv(256,(3,3),activation='relu',padding = 'same', strides=(1,1)) (model)
    model = conv(256,(3,3),activation='relu',padding = 'same', strides=(1,1)) (model)
    model = conv(256,(3,3),activation='relu',padding = 'same', strides=(1,1)) (model)
    model = maxpool(pool_size=(2,2),strides=(2,2),padding='same')(model)
    print("3 x CONV  of 256 filters  and (3,3) kernel size...")
    print("Max pool of (2,2) kernel size...")
    model = flt()(model)
    print("[INFO] flatten the previous layer...")
    
    print("[INFO] constructing head layer to ouput the predicted bounding box coordinates...")
    
    bboxhead = Dense(128,activation = "relu")(model)
    softmaxhead = Dropout(0.5) (softmaxhead)
    bboxhead = Dense(64,activation="relu")(bboxhead)
    bboxhead = Dense(32,activation="relu")(bboxhead)
    print("[INFO] 3 X FoolConnected Layers 128-64-32 ")
    bboxhead = Dense(4,activation="sigmoid",name="bounding box")(bboxhead)
    print("[INFO] output bounding box Layer of 4 nodes(xmin,ymin,xmax,ymax)")
    
    print("[INFO] constructing head layer to ouput the predicted class label...")
    
    softmaxhead = Dense(512,activation="relu")(model)
    softmaxhead = Dropout(0.5) (softmaxhead)
    print("[INFO] FoolConnected Layer of 512 nodes + Dropout of 0.5 ")
    softmaxhead = Dense(256,activation="relu")(softmaxhead)
    softmaxhead = Dropout(0.5) (softmaxhead)
    print("[INFO] FoolConnected Layer of 256 nodes + Dropout of 0.5 ")
    softmaxhead = Dense(nbClasses,activation="sotfmax",name="class_label")(softmaxhead)
    print("[INFO] output class label Layer of "+str(nbClasses)+" nodes ")
    
    FinalModel = Model(inputs = model,outputs = (bboxhead,softmaxhead))
    return FinalModel



In [ ]:
def compile_Model(model):
    losses={'class_label': 'categorical_crossentropy',
            'bounding box': 'mean_squared_error'}
    lossweights={'class_label': 1.0,
            'bounding box': 1.0}
    opt = Adam(lr=INIT_LR)
    model.compile(loss = losses,optimizer=opt,metrics=["accuracy"],loss_weights=lossweights)
    print(model.compile)
    return model


def Fit_Model(model,trainImages,testImages,Trainlabels,Testlabels,TrainBboxs,TestBboxs):
    trainTargets = {"class_label": Trainlabels,
                "bounding box": TrainBboxs}
    testTargets = {"class_label": Testlabels,
                "bounding box": TestBboxs}
    print("[INFO] training model...")    
    H = model.fit(trainImages,trainTargets,validation_data=(testImages,testTargets),batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,verbose =1)
    #verbose =1 => will show you an animated progress bar: 
    # [==============]
    #verbose = 0  => will show you nothing (silent)
    #verbose = 2 => will just mention the number of epoch :
    # Epoch 1/10
    print("[INFO] saving object detector model...")
    model.save(MODEL_PATH,save_format="h5")
    
    Loss_names = ['loss','class_label_loss','bounding box_loss']
    N= np.arrange(NUM_EPOCHS)
    plt.style.use("ggplot")
    (fig,ax)=plt.subplots(3,1,figsize=(13,13))
    for(i,l) in enumerate(Loss_names):
        title = "Loss for {}".format(l) if l != "loss" else"Total loss"
        ax[i].set_title(title)
        ax[i].set_xlabel("Epoch")
        ax[i].set_ylabel("loss")
        ax[i].plot(N,H.history[l],label = l)
        ax[i].plot(N,H.history["val_"+l],label = "val_"+l)
        ax[i].legend()
    plt.tight_layout()
    plot_path = os.path.join(PLOTS_PATH, "losses.png")
    plt.savefig(plot_path)
    plt.close()
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(N, H.history["class_label_accuracy"],label="class_label_train_acc")
    return H